In [1]:
import csv
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
import unicodedata
from datetime import datetime


In [2]:

# 1. Definir la URL base y las rutas de los archivos de entrada y salida
url_base = "https://www.inmuebles24.com/departamentos-en-renta-en-"
csv_colonias_filename = "/Users/renataramirez/Documents/mlops-repo/data/raw/CP/CPdescarga - Distrito_Federal.csv"


In [3]:

# Obtener solo la fecha actual para crear una carpeta de versionado
timestamp = datetime.now().strftime("%Y-%m-%d")
output_directory = f"/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/por_municipio_{timestamp}"


In [4]:

# 2. Crear listas para almacenar los datos
municipios_sin_resultados = []


In [5]:

def normalizar_nombre(nombre):
    """
    Normaliza el nombre (de municipio o colonia) para ser usado en la URL y en el nombre del archivo.
    - Convierte a minúsculas.
    - Elimina acentos y caracteres especiales.
    - Reemplaza espacios por guiones.
    """
    # Convertir a minúsculas
    nombre = nombre.lower()
    # Eliminar acentos
    nombre = unicodedata.normalize('NFKD', nombre).encode('ascii', 'ignore').decode('utf-8')
    # Reemplazar espacios y caracteres no alfanuméricos por guiones
    nombre = re.sub(r'[\s\W]+', '-', nombre)
    return nombre


In [6]:

def leer_municipios_csv(filepath):
    """
    Lee el archivo CSV y extrae los nombres únicos de los municipios.
    """
    municipios = []
    try:
        with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if 'D_mnpio' in row and row['D_mnpio']:
                    municipios.append(row['D_mnpio'].strip())
    except FileNotFoundError:
        print(f"Error: El archivo '{filepath}' no se encontró.")
        return None
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo CSV: {e}")
        return None
    
    return sorted(list(set(municipios)))


In [7]:

def scrape_current_page(driver, nombre_municipio):
    """
    Extrae los datos de todas las propiedades en la página actual.
    """
    inmuebles_en_pagina = []
    
    try:
        listings = driver.find_elements(By.CSS_SELECTOR, '.postingCardLayout-module__posting-card-layout')
        
        for listing in listings:
            inmueble = {}
            try:
                title_container = listing.find_element(By.CSS_SELECTOR, '.postingCard-module__posting-description')
                inmueble['Título'] = title_container.text.strip()
                title_element = title_container.find_element(By.TAG_NAME, 'a')
                inmueble['Enlace'] = title_element.get_attribute('href')
            except Exception:
                inmueble['Título'] = 'No disponible'
                inmueble['Enlace'] = 'No disponible'

            try:
                price_element = listing.find_element(By.CSS_SELECTOR, 'div[data-qa="POSTING_CARD_PRICE"]')
                inmueble['Precio'] = price_element.text.strip()
            except Exception:
                inmueble['Precio'] = 'No disponible'

            try:
                address_element = listing.find_element(By.CSS_SELECTOR, '.postingLocations-module__location-block')
                inmueble['Dirección'] = address_element.text.strip()
            except Exception:
                inmueble['Dirección'] = 'No disponible'

            try:
                features = listing.find_elements(By.CSS_SELECTOR, 'span.postingMainFeatures-module__posting-main-features-listing')
                features_list = [feature.text.strip() for feature in features]
                inmueble['Características'] = ', '.join(features_list)
            except Exception:
                inmueble['Características'] = 'No disponible'
            
            inmueble['Municipio de Origen'] = nombre_municipio
            
            inmuebles_en_pagina.append(inmueble)
    except Exception as e:
        print(f"Error al extraer datos de la página: {e}")
    
    return inmuebles_en_pagina


In [8]:

# --- Inicio del script principal ---

# 3. Leer la lista de municipios del archivo CSV
print("Leyendo la lista de municipios desde el archivo...")
lista_municipios = leer_municipios_csv(csv_colonias_filename)

if not lista_municipios:
    print("No se pudo obtener la lista de municipios. Terminando el script.")
    exit()

print(f"Se encontraron {len(lista_municipios)} municipios únicos para buscar.")


Leyendo la lista de municipios desde el archivo...
Se encontraron 16 municipios únicos para buscar.


In [9]:

# 4. Configurar el WebDriver de Selenium
try:
    print("\nIniciando el navegador web sin detección...")
    driver = uc.Chrome(options=uc.ChromeOptions(), timeout=240)
    driver.maximize_window()
except WebDriverException as e:
    print(f"Error al iniciar el navegador: {e}")
    print("Asegúrate de tener la librería 'undetected-chromedriver' instalada.")
    exit()



Iniciando el navegador web sin detección...


In [10]:

try:
    # Asegurar que el directorio de salida exista
    os.makedirs(output_directory, exist_ok=True)
    
    # 5. Iterar a través de cada municipio y realizar el scraping
    for index, nombre_municipio in enumerate(lista_municipios):
        normalized_name = normalizar_nombre(nombre_municipio)
        
        municipio_filename = os.path.join(output_directory, f"{normalized_name}.csv")
        if os.path.exists(municipio_filename):
            print(f"Archivo para '{nombre_municipio}' ya existe. Saltando este municipio. (van {index + 1} de {len(lista_municipios)}, faltan {len(lista_municipios) - (index + 1)})")
            continue
        
        print(f"\n--- Buscando en municipio: {nombre_municipio} (van {index + 1} de {len(lista_municipios)}, faltan {len(lista_municipios) - (index + 1)}) ---")

        # Lista para almacenar todos los datos de este municipio
        inmuebles_por_municipio = []
        
        # URL inicial
        url_to_visit = f"{url_base}{normalized_name}.html"
        
        while True:
            print(f"Buscando en la URL: {url_to_visit}")

            try:
                driver.get(url_to_visit)
                wait = WebDriverWait(driver, 10)
                
                # Esperar a que el contenedor de resultados o el mensaje de no resultados aparezca
                wait.until(EC.any_of(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.postingsList-module__postings-container')),
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.emptyResults-module__empty-results'))
                ))

                # Verificar si la página tiene un mensaje de "no hay resultados"
                if len(driver.find_elements(By.CSS_SELECTOR, '.emptyResults-module__empty-results')) > 0:
                    print("La página no tiene resultados. Finalizando la paginación.")
                    break
                
                # Scrapeamos los datos de la página actual
                current_page_data = scrape_current_page(driver, nombre_municipio)
                
                if not current_page_data:
                    # Si el scraper no encuentra listings pero tampoco el mensaje de no resultados,
                    # asumimos que la paginación ha terminado.
                    print("No se encontraron propiedades en la página. Finalizando la paginación.")
                    break
                
                inmuebles_por_municipio.extend(current_page_data)
                
                print(f"Página scrapeada con éxito. Total de propiedades hasta ahora: {len(inmuebles_por_municipio)}")
                
                # Intentar encontrar el botón de "Siguiente" y obtener su href
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'a[data-qa="PAGING_NEXT"]')
                    # Actualizar la URL para la siguiente iteración
                    url_to_visit = next_button.get_attribute('href')
                    time.sleep(2) # Pausa para cargar la siguiente página
                except NoSuchElementException:
                    print("Botón 'Siguiente' no encontrado. Fin de la paginación.")
                    break # El botón no existe, hemos llegado a la última página
            
            except TimeoutException:
                print("Se superó el tiempo de espera. Probablemente no existen más páginas.")
                break
            except Exception as e:
                print(f"Ocurrió un error inesperado al procesar la URL '{url_to_visit}': {e}")
                break

            # --- FIN DE LA LÓGICA DE PAGINACIÓN ---

        if inmuebles_por_municipio:
            with open(municipio_filename, 'w', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['Título', 'Enlace', 'Precio', 'Dirección', 'Características', 'Municipio de Origen']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(inmuebles_por_municipio)
            
            print(f"Datos de {len(inmuebles_por_municipio)} propiedades guardados en '{municipio_filename}'.")
        else:
            print(f"No se encontraron propiedades en ninguna página para el municipio '{nombre_municipio}'.")
            municipios_sin_resultados.append(nombre_municipio)
        
    
    # 9. Mostrar la lista de municipios que no tuvieron resultados
    if municipios_sin_resultados:
        print("\nLos siguientes municipios no arrojaron resultados:")
        for municipio in sorted(municipios_sin_resultados):
            print(f"- {municipio}")
    else:
        print("\n¡Todos los municipios procesados tuvieron al menos una propiedad!")

except Exception as e:
    print(f"Ocurrió un error inesperado durante la ejecución principal: {e}")
finally:
    # 10. Cerrar el navegador al finalizar
    print("\nCerrando el navegador.")
    driver.quit()

print("\nProceso de scraping por municipios completado.")



--- Buscando en municipio: Azcapotzalco (van 1 de 16, faltan 15) ---
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-azcapotzalco.html
Página scrapeada con éxito. Total de propiedades hasta ahora: 30
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-azcapotzalco-pagina-2.html
Página scrapeada con éxito. Total de propiedades hasta ahora: 60
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-azcapotzalco-pagina-3.html
Página scrapeada con éxito. Total de propiedades hasta ahora: 90
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-azcapotzalco-pagina-4.html
Página scrapeada con éxito. Total de propiedades hasta ahora: 91
Botón 'Siguiente' no encontrado. Fin de la paginación.
Datos de 91 propiedades guardados en '/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/por_municipio_2025-09-14/azcapotzalco.csv'.

--- Buscando en municipio: Benito Juárez (van 2 de 16, faltan 14) ---
Bus